# Toronto Dwellings Analysis

In this assignment, you will perform fundamental analysis for the Toronto dwellings market to allow potential real estate investors to choose rental investment properties.

In [1]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact


In [2]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("MAPBOX")

## Load Data

In [3]:
# Read the census data into a Pandas DataFrame
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")
to_data.head()

- - - 

## Dwelling Types Per Year

In this section, you will calculate the number of dwelling types per year. Visualize the results using bar charts and the Pandas plot function. 

**Hint:** Use the Pandas `groupby` function.

**Optional challenge:** Plot each bar chart in a different color.

In [4]:
# Select the dwelling type columns
dwelling_types = [
    'single_detached_house',
    'apartment_five_storeys_plus',
    'movable_dwelling',
    'semi_detached_house',
    'row_house',
    'duplex',
    'apartment_five_storeys_less',
    'other_house'
]
# Calculate the sum number of dwelling types units per year (hint: use groupby)
dwelling_sum = to_data[dwelling_types].groupby([to_data.index]).sum()
dwelling_sum

In [5]:
# Save the dataframe as a csv file
dwelling_sum.to_csv('Data/dwelling_sum.csv')

In [6]:
# Helper create_bar_chart function
def create_bar_chart(data, title, xlabel, ylabel, color):
    """
    Create a barplot based in the data argument.
    """
    return data.hvplot.bar(
        title=title,
        xlabel=xlabel,
        ylabel=ylabel,
        color=color,
        rot=90,
        yformatter='%.0f'
    )

In [7]:
# Set bar chart colors
colors = ['red', 'blue', 'orange', 'pink']
# Create a bar chart per year to show the number of dwelling types
bar_charts = [
    create_bar_chart(
        dwelling_sum.loc[year],     
        f'Dwelling Types in Toronto in {year}',
        str(year),
        "Dwelling Type Units",
        color,
    )
    for (year, color) in zip(dwelling_sum.index, colors)
]
# Plot the charts on a 2x2 grid
pane_dwelling_types = pn.Column(
    "# Dwelling Types in Toronto",
    pn.Row(*bar_charts[:2]),
    pn.Row(*bar_charts[2:]),
)
pane_dwelling_types

- - - 

## Average Monthly Shelter Costs in Toronto Per Year

In this section, you will calculate the average monthly shelter costs for owned and rented dwellings and the average house value for each year. Plot the results as a line chart.

**Optional challenge:** Plot each line chart in a different color.

In [8]:
# Calculate the average monthly shelter costs for owned and rented dwellings
shelter_cols = ['shelter_costs_owned', 'shelter_costs_rented']
df_shelter_costs = to_data[shelter_cols].groupby([to_data.index]).mean()
df_shelter_costs

In [9]:
# Helper create_line_chart function
def create_line_chart(data, title, xlabel, ylabel, color):
    """
    Create a line chart based in the data argument.
    """
    return data.hvplot.line(
        title=title,
        xlabel=xlabel,
        ylabel=ylabel,
        color=color,
        yformatter='%.0f'
    )

In [10]:
# Create two line charts, one to plot the monthly shelter costs for owned dwellings and other for rented dwellings per year

# Line chart for owned dwellings
line_owned = create_line_chart(
    df_shelter_costs['shelter_costs_owned'],
    title='Average Monthly Shelter Costs for Owned Dwellings in Toronto',
    xlabel='Year',
    ylabel='Avg Monthly Shelter Costs',
    color='blue',
)

# Line chart for rented dwellings
line_rented = create_line_chart(
    df_shelter_costs['shelter_costs_rented'],
    title='Average Monthly Shelter Costs for Rented Dwellings in Toronto',
    xlabel='Avg Monthly Shelter Costs',
    ylabel='Year',
    color='orange',
)
pane_shelter_costs = pn.Column(
    line_owned,
    line_rented,
)
pane_shelter_costs

## Average House Value per Year

In this section, you want to determine the average house value per year. An investor may want to understand better the sales price of the rental property over time. For example, a customer will want to know if they should expect an increase or decrease in the property value over time so they can determine how long to hold the rental property. You will visualize the `average_house_value` per year as a line chart.

In [11]:
# Calculate the average house value per year
df_avg_house_value = to_data['average_house_value'].groupby([to_data.index]).mean()
df_avg_house_value.head()

In [12]:
# Plot the average house value per year as a line chart
line_avg_house_value = create_line_chart(
    df_avg_house_value,
    title='Average House Value in Toronto',
    xlabel='Year',
    ylabel='Avg. House Value',
    color='blue',
)
line_avg_house_value

- - - 

## Average House Value by Neighbourhood

In this section, you will use `hvplot` to create an interactive visualization of the average house value with a dropdown selector for the neighbourhood.

**Hint:** It will be easier to create a new DataFrame from grouping the data and calculating the mean house values for each year and neighbourhood.

In [13]:
# Create a new DataFrame with the mean house values by neighbourhood per year
df_house_value_neighbourhood = to_data[['neighbourhood', 'average_house_value']]
df_house_value_neighbourhood.head()

In [14]:
# Use hvplot to create an interactive line chart of the average house value per neighbourhood
# The plot should have a dropdown selector for the neighbourhood
interact(
    lambda neighbourhood: create_line_chart(
        df_house_value_neighbourhood[df_house_value_neighbourhood['neighbourhood'] == neighbourhood],
        title=f'neighbourhood: {neighbourhood}',
        xlabel='Year',
        ylabel='Avg. House Value',
        color='blue',
    ),
    neighbourhood=df_house_value_neighbourhood['neighbourhood'].unique()
)

## Number of Dwelling Types per Year

In this section, you will use `hvplot` to create an interactive visualization of the average number of dwelling types per year with a dropdown selector for the neighbourhood.

In [15]:
# Fetch the data of all dwelling types per year
df_dwelling_types = to_data[dwelling_types + ['neighbourhood']].groupby(['neighbourhood', to_data.index]).sum()
df_dwelling_types.head()

In [16]:
# Use hvplot to create an interactive bar chart of the number of dwelling types per neighbourhood
# The plot should have a dropdown selector for the neighbourhood
interact(
    lambda neighbourhood: df_dwelling_types.loc[neighbourhood].hvplot.bar(
        title=f'neighbourhood: {neighbourhood}',
        xlabel= 'Year',
        ylabel='Dwelling Type Units',
        rot=90,
        yformatter='%.0f',
        height=500,
    ),
    neighbourhood=df_dwelling_types.index.levels[0]
)

- - - 

## The Top 10 Most Expensive Neighbourhoods

In this section, you will need to calculate the house value for each neighbourhood and then sort the values to obtain the top 10 most expensive neighbourhoods on average. Plot the results as a bar chart.

In [17]:
# Getting the data from the top 10 expensive neighbourhoods
df_top_10_expensive_neighbourhoods = (
    # select neighbourhood and average house value columns
    to_data[['neighbourhood', 'average_house_value']]
        # group by neighbourhood
        .groupby(['neighbourhood'])
        # calculate the mean
        .mean()
        # sort the values with the most expensive first
        .sort_values(by=['average_house_value'], ascending=False)
        # select only the top 10 (most expensive)
        .iloc[:10]
)

In [18]:
# Plotting the data from the top 10 expensive neighbourhoods
df_top_10_expensive_neighbourhoods.hvplot.bar(
    title='Top 10 Expensive Neighbourhoods in Toronto',
    xlabel='Neighbourhood',
    ylabel='Avg. House Value',
    height=500,
    rot=90,
    yformatter='%.0f',
)

- - - 

## Neighbourhood Map

In this section, you will read in neighbourhoods location data and build an interactive map with the average house value per neighbourhood. Use a `scatter_mapbox` from Plotly express to create the visualization. Remember, you will need your Mapbox API key for this.

### Load Location Data

In [19]:
# Load neighbourhoods coordinates data
file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path, index_col='neighbourhood')
df_neighbourhood_locations.head()

### Data Preparation

You will need to join the location data with the mean values per neighbourhood.

1. Calculate the mean values for each neighbourhood.

2. Join the average values with the neighbourhood locations.

In [20]:
# Calculate the mean values for each neighborhood
df_mean_value_neighbourhoods = (
    # select neighbourhood and average house value columns
    to_data[['neighbourhood', 'average_house_value']]
        # group by neighbourhood
        .groupby(['neighbourhood'])
        # calculate the mean
        .mean()
)
df_mean_value_neighbourhoods.head()

In [21]:
# Join the average values with the neighbourhood locations
df_neighbourhood_locations_mean = df_neighbourhood_locations.join(df_mean_value_neighbourhoods)
df_neighbourhood_locations_mean.head()

### Mapbox Visualization

Plot the average values per neighbourhood using a Plotly express `scatter_mapbox` visualization.

In [22]:
# Create a scatter mapbox to analyze neighbourhood info
map_plot = px.scatter_mapbox(
    df_neighbourhood_locations_mean,
    lat='lat',
    lon='lon',
    size='average_house_value',
    color='average_house_value',
    zoom=9,
    mapbox_style='carto-positron',
)

# Display the map
map_plot.show()

- - -

## Cost Analysis - Optional Challenge

In this section, you will use Plotly express to a couple of plots that investors can interactively filter and explore various factors related to the house value of the Toronto's neighbourhoods. 

### Create a bar chart row facet to plot the average house values for all Toronto's neighbourhoods per year

In [23]:
fig = px.bar(
    # Select neighbourhood and average_house_value columns
    to_data[['neighbourhood', 'average_house_value']],
    # Set plot title
    title='Average House Values in Toronto per Neighbourhood',
    # column of x values
    x='neighbourhood',
    # column of y values
    y='average_house_value',
    # Set color scale according to average house value
    color='average_house_value',
    # group data by year
    facet_row=to_data.index,
    # Display average_house_value as 'Avg. House Value'
    labels={
        'average_house_value': 'Avg. House Value',
    },
    # Set height of plot
    height=1000,
)
fig.show()

### Create a sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year

In [24]:
# Fetch the data from all expensive neighbourhoods per year.
# Select neighbourhood and average house value columns
df_top_10_expensive_neighbourhoods_year = to_data[['neighbourhood', 'average_house_value']]
# Select the top 10 most expensive neighbourhoods
df_top_10_expensive_neighbourhoods_year = df_top_10_expensive_neighbourhoods_year[to_data['neighbourhood'].isin(df_top_10_expensive_neighbourhoods.index)]
# Reset index and make the year a column
df_top_10_expensive_neighbourhoods_year.reset_index(inplace=True)
df_top_10_expensive_neighbourhoods_year.head()

In [25]:
# Create the sunburst chart
fig = px.sunburst(
    df_top_10_expensive_neighbourhoods_year,
    title='Costs Analysis of Most Expensive Neighbourhoods in Toronto per Year',
    path=['year', 'neighbourhood'],
    values='average_house_value',
    color='average_house_value',
    labels={
        'average_house_value': 'Avg. House Value',
    },
)
fig.show()